In [1]:
from urllib.parse import quote_plus    # 한글 텍스트를 퍼센트 인코딩으로 변환
from selenium import webdriver    # 라이브러리에서 사용하는 모듈만 호출
from selenium.webdriver.common.by import By   # 해당 태그가 존재하는지 확인 -> # 사용 예시: (By.CLASS_NAME, 'list_title')
from selenium.webdriver.support.ui import WebDriverWait   # 해당 태그를 기다림
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException    # 태그가 없는 예외 처리
import time
import pandas as pd

In [2]:
_input = input('''-월--일, -월, 이번주, 이번주말 중 선택하여 입력해주세요.
                                 (-은 숫자 입력, 이번년도만 가능) : ''')
user_input = quote_plus(_input)

url = f'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&query={user_input}%20%EC%97%B0%EA%B7%B9%20%EA%B3%B5%EC%97%B0'
user_input

-월--일, -월, 이번주, 이번주말 중 선택하여 입력해주세요.
                                 (-은 숫자 입력, 이번년도만 가능) : 12월27일


'12%EC%9B%9427%EC%9D%BC'

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
options.add_argument('disable-gpu')    # GPU 사용 안함
options.add_argument('lang=ko_KR')    # 언어 설정
chromedriver = '/Users/jihun/CrawlingProject/helpers/chromedriver'
driver = webdriver.Chrome(chromedriver, options = options)

driver.get(url)

In [4]:
try :
    element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'list_title')))

    pageNum = int(driver.find_element_by_class_name('_totalCount').text)

    theater_list = []

    count = 1
    
    for i in range(1, pageNum+1) :
        theater_data = driver.find_elements_by_class_name('list_title')
        img_data = driver.find_elements_by_class_name('list_thumb')

        for theater in theater_data :
            theater_list.append(theater.text.split('\n'))

        for img in img_data :
            img.screenshot(f'./images/{count}.png')
            count += 1

        driver.find_element_by_xpath("/html/body/div[3]/div[2]/div/div[1]/section[1]/div/div[2]/div/div[1]/div[4]/div[2]/div/a[2]").click()
        time.sleep(2)

except TimeoutException :  # 예외 처리
    print('해당 페이지에 연극 정보가 존재하지 않습니다.')

finally:    # 정상, 예외 둘 중 하나여도 반드시 실행
    driver.quit()

In [5]:
theater_list

[['한뼘사이', '17.03.01.~오픈런', '서연아트홀'],
 ['앙리할아버지와 나 - 서울', '20.12.03.~21.02.14.', '예스24스테이지 1관'],
 ['핫식스', '18.08.02.~오픈런', '대학로 키득키득아트홀'],
 ['오마이갓', '17.07.01.~오픈런', '봄날아트홀 2관'],
 ['음악극 [올드위키드송]', '20.12.08.~21.02.14.', '예스24스테이지 3관'],
 ['올모스트메인', '20.12.25.~21.02.14.', '대학로티오엠'],
 ['<연극열전8> 네 번째 작품 <킹스 스피치>', '20.11.28.~21.02.07.', '대학로아트원씨어터 2관'],
 ['오백에 삼십', '18.09.04.~오픈런', '대학로 아트포레스트 1관'],
 ['너의 목소리가 들려', '18.12.06.~오픈런', '봄날아트홀'],
 ['아날로그 정통 체력극 <신바람 난 삼대>', '20.12.23.~21.03.01.', '대학로아트포레스트2관'],
 ['연애하기 좋은 날 - 서울', '19.10.03.~오픈런', '아티스탄홀'],
 ['나의 PS 파트너', '18.03.08.~오픈런', '대학로 마루아트홀'],
 ['옥탑방 고양이 - 서울', '10.04.06.~오픈런', '틴틴홀'],
 ['사랑에 스치다', '20.12.16.~20.12.27.', '스카이시어터 2관'],
 ['너에게로 또 다시 - 광주', '20.11.11.~21.01.10.', '상무지구 기분좋은극장'],
 ['택시 안에서', '18.10.12.~오픈런', '해바라기 소극장'],
 ['헬로우 미스미스터 - 대구', '20.11.26.~21.01.03.', '문화예술전용극장CT'],
 ['러브앤전쟁', '20.12.11.~21.02.28.', '두레홀'],
 ['산악기상관측', '20.12.24.~20.12.27.', '연우소극장'],
 ['행오버 - 서울', '20.03.16.~오픈런', '해피씨어터'],
 ['늘근도둑 이야기 - 서울', 

In [6]:
# for theater in theater_list :
#     col_4 = theater[1].split('~')
#     theater.append()
#     theater.append(col_4[1])

# for i in range(len(theater_list)) :
#     if theater_list[i][4] == '오픈런' :
#         theater_list[i][4] = '50.01.01'
#         theater_list[i].append('True')   # theater_list[i][5]가 아직 없기 때문에 theater_list에 5번 인덱스를 append해야 한다.
#     else :
#         theater_list[i].append('False')

In [7]:
df = pd.DataFrame(data = theater_list, columns = ['제목', '기간', '장소'])

In [8]:
start_day = []

for i in range(len(theater_list)):
    start = theater_list[i][1].split('~')[0]
    start_day.append(start)

start_series = pd.Series(data = start_day)

df['개막일'] = start_series

In [9]:
close_day = []


for i in range(len(theater_list)) :
    try :
        close = theater_list[i][1].split('~')[1]
        close_day.append(close)
    except IndexError :
        close_day.append('NaN')
        
close_day

['오픈런',
 '21.02.14.',
 '오픈런',
 '오픈런',
 '21.02.14.',
 '21.02.14.',
 '21.02.07.',
 '오픈런',
 '오픈런',
 '21.03.01.',
 '오픈런',
 '오픈런',
 '오픈런',
 '20.12.27.',
 '21.01.10.',
 '오픈런',
 '21.01.03.',
 '21.02.28.',
 '20.12.27.',
 '오픈런',
 '오픈런',
 '21.01.10.',
 '21.01.03.',
 '20.12.27.',
 '오픈런',
 '오픈런',
 '20.12.27.',
 '오픈런',
 '21.01.03.',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '21.01.03.',
 '20.12.29.',
 '오픈런',
 '오픈런',
 '20.12.27.',
 '20.12.31.',
 '오픈런',
 '20.12.27.',
 '오픈런',
 '21.01.03.',
 '20.12.31.',
 '21.01.03.',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '21.03.01.',
 '오픈런',
 '21.01.03.',
 '20.12.27.',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '20.12.31.',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런',
 '오픈런'

In [10]:
close_series = pd.Series(data = close_day)
df['폐막일'] = close_series

In [11]:
df.index = df.index + 1

In [12]:
df

,제목,기간,장소,개막일,폐막일
1,한뼘사이,17.03.01.~오픈런,서연아트홀,17.03.01.,오픈런
2,앙리할아버지와 나 - 서울,20.12.03.~21.02.14.,예스24스테이지 1관,20.12.03.,21.02.14.
3,핫식스,18.08.02.~오픈런,대학로 키득키득아트홀,18.08.02.,오픈런
4,오마이갓,17.07.01.~오픈런,봄날아트홀 2관,17.07.01.,오픈런
5,음악극 [올드위키드송],20.12.08.~21.02.14.,예스24스테이지 3관,20.12.08.,21.02.14.
...,...,...,...,...,...
135,스위치 - 서울,17.07.18.~오픈런,아루또 소극장,17.07.18.,오픈런
136,ANIMA,20.12.08.~20.12.27.,파랑씨어터,20.12.08.,20.12.27.
137,[온라인 극장] 동양극장 2020,20.12.10.~20.12.28.,None,20.12.10.,20.12.28.
138,엄마는 오십에 바다를 발견했다,20.12.21.~20.12.27.,허수아비 소극장,20.12.21.,20.12.27.


In [13]:
df['오픈런'] = 'False'
condition = df['폐막일'] == '오픈런'
df.loc[condition, '폐막일'] = '50.01.01'
condition2 = df['폐막일'] == '50.01.01'
df.loc[condition2, '폐막일'] = '50.01.01.'

In [14]:
df

,제목,기간,장소,개막일,폐막일,오픈런
1,한뼘사이,17.03.01.~오픈런,서연아트홀,17.03.01.,50.01.01.,False
2,앙리할아버지와 나 - 서울,20.12.03.~21.02.14.,예스24스테이지 1관,20.12.03.,21.02.14.,False
3,핫식스,18.08.02.~오픈런,대학로 키득키득아트홀,18.08.02.,50.01.01.,False
4,오마이갓,17.07.01.~오픈런,봄날아트홀 2관,17.07.01.,50.01.01.,False
5,음악극 [올드위키드송],20.12.08.~21.02.14.,예스24스테이지 3관,20.12.08.,21.02.14.,False
...,...,...,...,...,...,...
135,스위치 - 서울,17.07.18.~오픈런,아루또 소극장,17.07.18.,50.01.01.,False
136,ANIMA,20.12.08.~20.12.27.,파랑씨어터,20.12.08.,20.12.27.,False
137,[온라인 극장] 동양극장 2020,20.12.10.~20.12.28.,None,20.12.10.,20.12.28.,False
138,엄마는 오십에 바다를 발견했다,20.12.21.~20.12.27.,허수아비 소극장,20.12.21.,20.12.27.,False


In [15]:
df['개막일'] = pd.to_datetime(df['개막일'], format='%y.%m.%d.')
df['폐막일'] = pd.to_datetime(df['폐막일'], format='%y.%m.%d.')
df

,제목,기간,장소,개막일,폐막일,오픈런
1,한뼘사이,17.03.01.~오픈런,서연아트홀,2017-03-01,2050-01-01,False
2,앙리할아버지와 나 - 서울,20.12.03.~21.02.14.,예스24스테이지 1관,2020-12-03,2021-02-14,False
3,핫식스,18.08.02.~오픈런,대학로 키득키득아트홀,2018-08-02,2050-01-01,False
4,오마이갓,17.07.01.~오픈런,봄날아트홀 2관,2017-07-01,2050-01-01,False
5,음악극 [올드위키드송],20.12.08.~21.02.14.,예스24스테이지 3관,2020-12-08,2021-02-14,False
...,...,...,...,...,...,...
135,스위치 - 서울,17.07.18.~오픈런,아루또 소극장,2017-07-18,2050-01-01,False
136,ANIMA,20.12.08.~20.12.27.,파랑씨어터,2020-12-08,2020-12-27,False
137,[온라인 극장] 동양극장 2020,20.12.10.~20.12.28.,None,2020-12-10,2020-12-28,False
138,엄마는 오십에 바다를 발견했다,20.12.21.~20.12.27.,허수아비 소극장,2020-12-21,2020-12-27,False


In [16]:
df.loc[df['폐막일'] == '2050-01-01', '오픈런'] = 'True'

In [17]:
df

,제목,기간,장소,개막일,폐막일,오픈런
1,한뼘사이,17.03.01.~오픈런,서연아트홀,2017-03-01,2050-01-01,True
2,앙리할아버지와 나 - 서울,20.12.03.~21.02.14.,예스24스테이지 1관,2020-12-03,2021-02-14,False
3,핫식스,18.08.02.~오픈런,대학로 키득키득아트홀,2018-08-02,2050-01-01,True
4,오마이갓,17.07.01.~오픈런,봄날아트홀 2관,2017-07-01,2050-01-01,True
5,음악극 [올드위키드송],20.12.08.~21.02.14.,예스24스테이지 3관,2020-12-08,2021-02-14,False
...,...,...,...,...,...,...
135,스위치 - 서울,17.07.18.~오픈런,아루또 소극장,2017-07-18,2050-01-01,True
136,ANIMA,20.12.08.~20.12.27.,파랑씨어터,2020-12-08,2020-12-27,False
137,[온라인 극장] 동양극장 2020,20.12.10.~20.12.28.,None,2020-12-10,2020-12-28,False
138,엄마는 오십에 바다를 발견했다,20.12.21.~20.12.27.,허수아비 소극장,2020-12-21,2020-12-27,False


In [18]:
print('데이터프레임화 작업이 완료 되었습니다.')

데이터프레임화 작업이 완료 되었습니다.


In [19]:
df.to_csv(f'theater_{_input}_df.csv', mode='w', encoding='utf-8-sig',
                   header=True, index=True)